In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers faiss-cpu groq
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer
model_bi = SentenceTransformer("deepvk/USER-bge-m3")
model_cross = CrossEncoder("DiTy/cross-encoder-russian-msmarco")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.34k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import faiss
import json
sentences = []
with open("/content/drive/MyDrive/QA/orig_data/chunks_7.json", "r") as f:
    sentences = json.load(f)
index = faiss.read_index("/content/drive/MyDrive/QA/orig_data/embeddings_flatIP_7.index")
metadata = {}
with open("/content/drive/MyDrive/QA/orig_data/metadata_7.json", "r") as f:
    metadata = json.load(f)

In [ ]:
query = """
В каких случаях лицо, не являющееся собственником имущества, имеет право на защиту своего владения даже против собственника?
"""
query_vector = model_bi.encode([query])
faiss.normalize_L2(query_vector)
distances, indices = index.search(query_vector, 200)
possible_answers = []
for i in range(len(indices[0])):
  possible_answers.append(sentences[indices[0][i]])
s = model_cross.rank(query, possible_answers)
context = ""
for i in range(5):
  meta = metadata[str(indices[0][s[i]["corpus_id"]])]
  context += f"Факт {str(i+1)}: {possible_answers[s[i]['corpus_id']]}. Источник:\nкнига - {meta['book']}\nномер статьи - {meta['article_num']}\nссылка на книгу - {meta['link']}\n"

print(context)

Факт 1: 2. До приобретения на имущество права собственности в силу приобретательной давности лицо, владеющее имуществом как своим собственным, имеет право на защиту своего владения против третьих лиц, не являющихся собственниками имущества, а также не имеющих прав на владение им в силу иного предусмотренного законом или договором основания.. Источник:
книга - Гражданский Кодекс КР Часть I
номер статьи - 265
ссылка на книгу - https://cbd.minjust.gov.kg/4/edition/7214/ru
Факт 2: 3. В случаях и в порядке, предусмотренных настоящим Кодексом, лицо может приобрести право собственности на имущество, не имеющее собственника, на имущество, собственник которого неизвестен, либо на имущество, от которого собственник отказался или на которое он утратил право собственности по иным основаниям.. Источник:
книга - Гражданский Кодекс КР Часть I
номер статьи - 251
ссылка на книгу - https://cbd.minjust.gov.kg/4/edition/7214/ru
Факт 3: Права, предусмотренные статьями 289-291 настоящего Кодекса, принадле

In [ ]:
from groq import Client
client = Client(api_key="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")

system_prompt = """
Ты Q&A ассистент по вопросам законов Кыргызской Республики (Кыргызстана).
Твоя задача - ответить на заданный вопрос на основе переданной тебе информации,
но пожалуйста не ссылайся на то, из какой это статьи или кодекса.
Если вопрос касается законов Кыргызской республики, но в контексте нет ответа,
то ответь, что ты не знаешь ответа на этот вопрос. А если же вопрос не касается
законов Кыргызской Республики, то ответь только то, что ты не знаешь, потому что
ты помощник по вопросам законов Кыргызской Республики!
Укажи исходя из какой книги ответ, и сошлись на номер статьи, в конце ответа приведи добавь ссылку на книгу.
Ответ обязательно должен быть на русском, если ты придумал его на другом языке, то потом обязательно переведи на русский.
"""

query_wrapper = f"""
На основе вот этой информации {context},
ответь на вопрос клиента {query},
не забудь, что ответ должен быть на русском.
"""

responce = client.chat.completions.create(
    model = "llama3-70b-8192",
    messages = [
        {"role" : "system", "content" : system_prompt},
        {"role" : "user", "content" : query_wrapper},
    ]
)

print(responce.choices[0].message.content)

Лицо, не являющееся собственником имущества, имеет право на защиту своего владения даже против собственника, если оно владеет имуществом на праве бессрочного пользования земельным участком, хозяйственного ведения, оперативного управления либо по иному основанию, предусмотренному законом или договором.

Источник: книга - Гражданский Кодекс КР Часть I, статья 294. 
https://cbd.minjust.gov.kg/4/edition/7214/ru


In [ ]:
from groq import Client
client = Client(api_key="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")

system_prompt = """
Ты Q&A ассистент по вопросам законов Кыргызской Республики (Кыргызстана).
Твоя задача - ответить на заданный вопрос на основе переданной тебе информации,
но пожалуйста не ссылайся на то, из какой это статьи или кодекса.
Если вопрос касается законов Кыргызской республики, но в контексте нет ответа,
то ответь, что ты не знаешь ответа на этот вопрос. А если же вопрос не касается
законов Кыргызской Республики, то ответь только то, что ты не знаешь, потому что
ты помощник по вопросам законов Кыргызской Республики!
Ответ обязательно должен быть на русском, если ты придумал его на другом языке, то потом обязательно переведи на русский.
"""

while True:
  query = input("Введите ваш вопрос: ")
  query_vector = model_bi.encode([query])
  faiss.normalize_L2(query_vector)
  distances, indices = index.search(query_vector, 200)
  possible_answers = []
  for i in range(len(indices[0])):
    possible_answers.append(sentences[indices[0][i]])
  s = model_cross.rank(query, possible_answers)
  context = ""
  for i in range(10):
    context += "Факт " + str(i+1) + ". " + possible_answers[s[i]["corpus_id"]] + "\n"

  query_wrapper = f"""
  На основе вот этой информации {context},
  ответь на вопрос клиента {query},
  не пиши номер факта, а только ответ,
  не забудь, что ответ должен быть на русском.
  """

  responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : system_prompt},
          {"role" : "user", "content" : query_wrapper},
      ]
  )

  print("Ответ:", responce.choices[0].message.content)

In [ ]:
from groq import Client
import pandas as pd
client = Client(api_key="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")

system_prompt = """
Ты Q&A ассистент по вопросам законов Кыргызской Республики (Кыргызстана).
Твоя задача - ответить на заданный вопрос на основе переданной тебе информации,
но пожалуйста не ссылайся на то, из какой это статьи или кодекса.
Если вопрос касается законов Кыргызской республики, но в контексте нет ответа,
то ответь, что ты не знаешь ответа на этот вопрос. А если же вопрос не касается
законов Кыргызской Республики, то ответь только то, что ты не знаешь, потому что
ты помощник по вопросам законов Кыргызской Республики!
Ответ обязательно должен быть на русском, если ты придумал его на другом языке, то потом обязательно переведи на русский.
"""

path_to_question = "/content/drive/MyDrive/QA/orig_data/test_questions.xlsx"
questions_df = pd.read_excel(path_to_question)
questions = questions_df["Question"].tolist()
answers = []

counter = 1
for query in questions:
  print(counter, query)
  counter += 1
  query_vector = model_bi.encode([query])
  faiss.normalize_L2(query_vector)
  distances, indices = index.search(query_vector, 200)
  possible_answers = []
  for i in range(len(indices[0])):
    possible_answers.append(sentences[indices[0][i]])
  s = model_cross.rank(query, possible_answers)
  context = ""
  for i in range(10):
    context += "Факт " + str(i+1) + ". " + possible_answers[s[i]["corpus_id"]] + "\n"
  query_wrapper = f"""
  На основе вот этой информации {context},
  ответь на вопрос клиента {query},
  не ссылайся на номера фактов,
  не забудь, что ответ должен быть на русском.
  """

  responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : system_prompt},
          {"role" : "user", "content" : query_wrapper},
      ]
  )
  print(responce.choices[0].message.content.strip())
  answers.append(responce.choices[0].message.content.strip())

res = pd.DataFrame({"question" : questions, "answer" : answers})
res.to_excel("answers.xlsx", index=False)


#ADDING MEMORY

In [ ]:
def reformat(query, response):
  return f"Вопрос: {query}\nОтвет: {response}"

In [ ]:
from groq import Client
client = Client(api_key="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")

system_prompt = """
Ты Q&A ассистент по вопросам законов Кыргызской Республики (Кыргызстана).
Твоя задача - ответить на заданный вопрос на основе переданной тебе информации,
но пожалуйста не ссылайся на то, из какой это статьи или кодекса.
Если вопрос касается законов Кыргызской республики, но в контексте нет ответа,
то ответь, что ты не знаешь ответа на этот вопрос. А если же вопрос не касается
законов Кыргызской Республики, то ответь только то, что ты не знаешь, потому что
ты помощник по вопросам законов Кыргызской Республики!
Ответ обязательно должен быть на русском, если ты придумал его на другом языке, то потом обязательно переведи на русский.
"""


history = ""

while True:
  query = input("Ваш вопрос: ")

  if len(history) > 0:
    history_system_prompt = f"""
    Ты мой ассистент, для переформулировки предложений если это требуется.
    У меня есть предыдущий вопрос и ответ от LLM. Я тебе даю новый вопрос от этого
    же клиента, и если чтобы понять текущий вопрос, я должен обратиться к предыдущему вопросу и ответу на него,
    то тебе надо переформулировать текущий вопрос, чтобы его смысл был понятен и без учета предыдущего вопроса - ответа.
    При этом не добавляй ничего лишнего!
    Допустим, текущий вопрос такой: 'Могу ли я покрасить машину?',
    а предыдущий вопрос и ответ на него такой: '
    - Сколько у Месси детей?
    - У Месси 3 детей.
    ', в этом случае никакой связи тут нет и твой ответ просто такой: 'Могу ли я покрасить машину?'.
    Второй пример:
    Текущий вопрос 'А расскажи о нем подробнее'
    Предыдущий вопрос и ответ: '
    - Кто автор книги "Война и мир"?
    - Автор книги "Война и мир" - Толстой
    ', тут видим четкую связь, что клиент хочет узнать подробную информацию о авторе данной книги, и
    твой ответ должен быть таким: 'А расскажи подробнее о Толстом'.
    Весь процесс должен быть на русском, и обратный ответ возвращай на русском!
    """

    history_query_wrapper = f"""
    Вот предыдущий вопрос - ответ: {history},
    если ты увидишь связь между этими сообщениями и этим: {query},
    то переформулируй '{query}' так, чтобы его смысл был понятен и без учета предыдущего вопроса - ответа.
    А если связи там нет, то просто верни '{query}'.
    Не пиши ничего лишнего, и ход своих мыслей тоже, напиши только переформулированный вопрос!
    """
    responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : history_system_prompt},
          {"role" : "user", "content" : history_query_wrapper},
      ]
  )

    query = responce.choices[0].message.content
    print("Переформулированный вопрос:", query)


  query_vector = model_bi.encode([query])
  faiss.normalize_L2(query_vector)
  distances, indices = index.search(query_vector, 200)
  possible_answers = []
  for i in range(len(indices[0])):
    possible_answers.append(sentences[indices[0][i]])
  s = model_cross.rank(query, possible_answers)
  context = ""
  for i in range(10):
    context += "Факт " + str(i+1) + ". " + possible_answers[s[i]["corpus_id"]] + "\n"

  query_wrapper = f"""
  На основе вот этой информации {context},
  ответь на вопрос клиента {query},
  не забудь, что ответ должен быть на русском.
  """

  responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : system_prompt},
          {"role" : "user", "content" : query_wrapper},
      ]
  )

  print("Ответ:", responce.choices[0].message.content)
  history = reformat(query, responce.choices[0].message.content)
  #Какие права имеет человек?
  #Можно ли считать имя неотъемлемой частью meiner личности, защищенной законом?

Ваш вопрос: Какие права есть у человека?
Ответ: У человека есть следующие права:

* Вещные права, такие как право собственности, право хозяйственного ведения, право оперативного управления, право бессрочного пользования земельным участком, право ограниченного пользования чужим недвижимым имуществом (сервитут) и другие;
* Право на охрану тайны личной жизни, включая тайну переписки, почтовых, телеграфных, электронных и иных сообщений, телефонных и иных переговоров, дневников, заметок, записок, интимной жизни, рождения, усыновления, врачебной или адвокатской тайны, тайны вкладов и т.д.;
* Право на неприкосновенность жилища;
* Право на уважение чести и достоинства личности;
* Право на представительство в суде;
* Право на владение, пользование и распоряжение имуществом;
* Право на имя, его использование и защиту;
* Право на предпринимательскую и иную деятельность;
* Право на создание юридических лиц;
* Право на совершение сделок и участие в обязательствах;
* Право на выбор места жительства;

KeyboardInterrupt: Interrupted by user

#RESULT

In [ ]:
system_prompt = """
Ты Q&A ассистент по вопросам законов Кыргызской Республики (Кыргызстана).
Твоя задача - ответить на заданный вопрос на основе переданной тебе информации,
но пожалуйста не ссылайся на то, из какой это статьи или кодекса.
Если вопрос касается законов Кыргызской республики, но в контексте нет ответа,
то ответь, что ты не знаешь ответа на этот вопрос. А если же вопрос не касается
законов Кыргызской Республики, то ответь только то, что ты не знаешь, потому что
ты помощник по вопросам законов Кыргызской Республики!
Укажи исходя из какой книги ответ, и сошлись на номер статьи, в конце ответа приведи добавь ссылку на книгу.
Ответ обязательно должен быть на русском, если ты придумал его на другом языке, то потом обязательно переведи на русский.
"""

history_system_prompt = """
Ты мой ассистент, для переформулировки предложений если это требуется.
У меня есть предыдущий вопрос и ответ от LLM. Я тебе даю новый вопрос от этого
же клиента, и если чтобы понять текущий вопрос, я должен обратиться к предыдущему вопросу и ответу на него,
то тебе надо переформулировать текущий вопрос, чтобы его смысл был понятен и без учета предыдущего вопроса - ответа.
При этом не добавляй ничего лишнего!
Допустим, текущий вопрос такой: 'Могу ли я покрасить машину?',
а предыдущий вопрос и ответ на него такой: '
- Сколько у Месси детей?
- У Месси 3 детей.
', в этом случае никакой связи тут нет и твой ответ просто такой: 'Могу ли я покрасить машину?'.
Второй пример:
Текущий вопрос 'А расскажи о нем подробнее'
Предыдущий вопрос и ответ: '
- Кто автор книги "Война и мир"?
- Автор книги "Война и мир" - Толстой
', тут видим четкую связь, что клиент хочет узнать подробную информацию о авторе данной книги,
твой ответ должен быть таким: 'А расскажи подробнее о Толстом'.
Весь процесс должен быть на русском, и обратный ответ возвращай на русском!
"""

history = ""

In [ ]:
from groq import Client
client = Client(api_key="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")

while True:
  query = input("Ваш вопрос: ")

  if len(history) > 0:
    history_query_wrapper = f"""
    Вот предыдущий вопрос - ответ: {history},
    если ты увидишь связь между этими сообщениями и этим: {query},
    то переформулируй '{query}' так, чтобы его смысл был понятен и без учета предыдущего вопроса - ответа.
    А если связи там нет, то просто верни '{query}'.
    Не пиши ничего лишнего, и ход своих мыслей тоже, напиши только переформулированный вопрос!
    """
    responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : history_system_prompt},
          {"role" : "user", "content" : history_query_wrapper},
      ]
    )

    query = responce.choices[0].message.content
    print("Переформулированный вопрос:", query)

  query_vector = model_bi.encode([query])
  faiss.normalize_L2(query_vector)
  distances, indices = index.search(query_vector, 200)
  possible_answers = []
  for i in range(len(indices[0])):
    possible_answers.append(sentences[indices[0][i]])
  s = model_cross.rank(query, possible_answers)
  context = ""
  for i in range(5):
    meta = metadata[str(indices[0][s[i]["corpus_id"]])]
    context += f"Факт {str(i+1)}: {possible_answers[s[i]['corpus_id']]}. Источник:\nкнига - {meta['book']}\nномер статьи - {meta['article_num']}\nссылка на книгу - {meta['link']}\n"

  query_wrapper = f"""
  На основе вот этой информации {context},
  ответь на вопрос клиента {query},
  не забудь, что ответ должен быть на русском.
  """

  responce = client.chat.completions.create(
      model = "llama3-70b-8192",
      messages = [
          {"role" : "system", "content" : system_prompt},
          {"role" : "user", "content" : query_wrapper},
      ]
  )

  print("Ответ:", responce.choices[0].message.content)
  history = reformat(query, responce.choices[0].message.content)


Ваш вопрос: какие права есть у человека?
Ответ: У человека есть следующие права:

* Право на охрану тайны личной жизни (тайны переписки, почтовых, телеграфных, электронных и иных сообщений, телефонных и иных переговоров, дневников, заметок, записок, интимной жизни, рождения, усыновления, врачебной или адвокатской тайны, тайны вкладов и т.п.)
* Право на неприкосновенность жилища (использование своего жилища по своему усмотрению в соответствии с его назначением и пресечение любых попыток вторжения в жилище помимо его воли, кроме случаев, предусмотренных законом)
* Право собственности (владеть, пользоваться и распоряжаться принадлежащим ему имуществом по своему усмотрению)
* Вещные права, такие как право хозяйственного ведения, право оперативного управления, право бессрочного пользования земельным участком, право ограниченного пользования чужим недвижимым имуществом (сервитут) и иные вещные права, на имущество лиц, не являющихся собственниками этого имущества в случаях, предусмотренных за

KeyboardInterrupt: Interrupted by user